## 전처리
### pycaret 기본 결측치 처리

In [1]:
# from pycaret.regression import *
# import pandas as pd
# import numpy as np

# data = pd.read_csv('/content/train.csv')
# data_MLM = data.drop('HLM', axis=1)
# data_MLM = setup(data = data,
#                  target = 'MLM',
#                  text_features=['SMILES'],
#                  ignore_features = ['id'],
#                  remove_outliers = True,
#                  normalize = True,
#                 #  transformation = True,
#                 #  transform_target = True,
#                 #  polynomial_features = True,
#                 #  feature_selection = True,
#                 #  pca = True, pca_components = 10,
#                  use_gpu = True,
#                  session_id = 123,
#                 #  profile = True
#                  )
# data_MLM

# dt_MLM = create_model('dt')

# # tune model
# tuned_dt_MLM = tune_model(dt_MLM, n_iter = 50, optimize = 'RMSE')

# # ensemble model
# ensemble_model(tuned_dt_MLM, choose_better = True)

# blender_soft = blend_models(compare_models(), method = 'soft', choose_better = True)

# blender_soft = blend_models(compare_models(), method = 'soft', choose_better = True)

# # train meta-model
# lightgbm = create_model('lightgbm')

# # stack models
# stacker = stack_models(compare_models(), meta_model = lightgbm)

# # default model
# print(dt_MLM)

# # tuned model
# print(tuned_dt_MLM)

# # ensemble model
# print(ensemble_model)

# # blended model
# print(blender_soft)

# # stack models
# print(stacker.final_estimator_)

In [2]:
# pip install "pycaret[full]"

In [3]:
from pycaret.regression import *
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')

# AlogP 결측치가 없는 데이터만 선택
data_no_missing = train_data[train_data['AlogP'].notnull()]

# PyCaret setup
reg_setup = setup(data_no_missing, 
                  target='AlogP', 
                  text_features=['SMILES'],
                  ignore_features=['id', 'MLM', "HLM"],
                  normalize = True,
                  verbose=True, 
                  session_id=123)

# 상위 3개 모델 선택
top_3_models = compare_models(n_select=3)

# 상위 모델 중 가장 성능이 좋은 모델 선택
best_model = top_3_models[0]

# AlogP 결측치가 있는 데이터만 선택
data_missing = train_data[train_data['AlogP'].isnull()]

# 예측하기 위한 특성 데이터 준비
features_for_prediction = data_missing.drop(columns=['id', 'AlogP', 'MLM', "HLM"])

# 결측치 예측
predicted_AlogP = predict_model(best_model, data=features_for_prediction)['prediction_label'].values

# 원래 데이터셋에 예측값 삽입
train_data.loc[train_data['AlogP'].isnull(), 'AlogP'] = predicted_AlogP

# 결과를 CSV 파일로 저장
train_data.to_csv('train_pycaret.csv', index=False)


,Description,Value
0,Session id,123
1,Target,AlogP
2,Target type,Regression
3,Original data shape,"(3496, 11)"
4,Transformed data shape,"(3496, 2635)"
5,Transformed train set shape,"(2447, 2635)"
6,Transformed test set shape,"(1049, 2635)"
7,Ignore features,3
8,Numeric features,6
9,Text features,1


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.2765,0.1947,0.4394,0.9236,0.1554,0.3554,0.0600
rf,Random Forest Regressor,0.2434,0.1975,0.4425,0.9225,0.1528,0.2866,0.0660
xgboost,Extreme Gradient Boosting,0.2725,0.1987,0.4439,0.9221,0.1540,0.3228,0.0690
catboost,CatBoost Regressor,0.2793,0.2018,0.4476,0.9207,0.1571,0.3159,0.2700
et,Extra Trees Regressor,0.2325,0.2070,0.4529,0.9188,0.1575,0.2667,0.0700
lar,Least Angle Regression,0.2849,0.2191,0.4661,0.9138,0.1583,0.3272,0.0660
omp,Orthogonal Matching Pursuit,0.3132,0.2992,0.5454,0.8819,0.1728,0.3060,0.0550
ada,AdaBoost Regressor,0.4680,0.3226,0.5673,0.8728,0.1908,0.6896,0.0590
dt,Decision Tree Regressor,0.2886,0.3403,0.5798,0.8666,0.1928,0.3628,0.0630
huber,Huber Regressor,0.5216,0.4780,0.6883,0.8106,0.2206,0.6759,0.0580


Processing:   0%|          | 0/87 [00:00<?, ?it/s]

In [4]:
# 테스트 데이터 로드
test_data = pd.read_csv('test.csv')

# 예측하기 위한 특성 데이터 준비 (테스트 데이터셋)
features_for_test_prediction = test_data[test_data['AlogP'].isnull()].drop(columns=['id', 'AlogP'])

# 테스트 데이터의 결측치 예측
predicted_AlogP_test = predict_model(best_model, data=features_for_test_prediction)['prediction_label'].values

# 테스트 데이터셋에 예측값 삽입
test_data.loc[test_data['AlogP'].isnull(), 'AlogP'] = predicted_AlogP_test

# 테스트 데이터의 결과를 CSV 파일로 저장
test_data.to_csv('test_pycaret.csv', index=False)


In [1]:
from pycaret.regression import *
import pandas as pd

# 데이터 로딩
data = pd.read_csv('train_pycaret.csv')

In [2]:
# MLM을 위한 setup
setup_data_MLM = setup(data = data,
                       target = 'MLM',
                       text_features=['SMILES'],
                       ignore_features = ['id', 'HLM'],
                       remove_outliers = True,
                       normalize = True,
                       # use_gpu = True,
                       session_id = 123,
                       verbose=True,
                       # transformation = True,
                       # transform_target = True,
                       # polynomial_features = True,
                       #  feature_selection = True,
                        # pca = True, pca_components = 10,
                       #  profile = True
                      )

,Description,Value
0,Session id,123
1,Target,MLM
2,Target type,Regression
3,Original data shape,"(3498, 11)"
4,Transformed data shape,"(3375, 2614)"
5,Transformed train set shape,"(2325, 2614)"
6,Transformed test set shape,"(1050, 2614)"
7,Ignore features,2
8,Numeric features,7
9,Text features,1


In [ ]:
# MLM 모델 학습, 튜닝, 블렌딩, 스택킹
top_5_MLM = compare_models(n_select=5, fold=10, sort='RMSE', turbo=False)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:37:51
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Automatic Relevance Determination


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
llar,Lasso Least Angle Regression,28.6291,1095.1963,33.0840,0.1475,1.7209,76.2438,0.1090
lasso,Lasso Regression,28.2944,1109.4055,33.2934,0.1363,1.7012,74.8626,0.1380
en,Elastic Net,28.9141,1135.5263,33.6861,0.1162,1.7129,74.7140,0.1080
lar,Least Angle Regression,28.2719,1149.3748,33.8833,0.1053,1.6857,74.2832,0.3190
br,Bayesian Ridge,30.2367,1173.3205,34.2443,0.0871,1.7587,79.6595,1.3440
omp,Orthogonal Matching Pursuit,30.0291,1494.6506,38.6077,-0.1640,1.6714,72.4014,0.1990
ridge,Ridge Regression,48.1804,4029.4165,63.2708,-2.1427,1.9849,127.3629,0.1520
lr,Linear Regression,2331619637798941.5000,32766179834592241686124274122752.0000,5636921454916205.0000,-25565785617239102937834192896.0000,19.0040,4859334927210162.0000,1.3990


Processing:   0%|          | 0/113 [00:00<?, ?it/s]

In [ ]:
top_5_MLM_10fold = compare_models(n_select=5, fold=10, sort='RMSE', turbo=False)

In [ ]:
tuned_MLM = [tune_model(model, optimize='RMSE', n_iter=10) for model in top_5_MLM]

In [7]:
blended_MLM = blend_models(tuned_MLM, optimize='RMSE', choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,27.8473,1264.2101,35.5557,0.0234,1.5544,36.3952
1,27.8183,1240.8844,35.2262,0.0196,1.3437,13.7370
2,26.0536,1161.7394,34.0843,0.0361,1.5232,32.5851
3,25.3501,1075.4820,32.7945,0.1807,1.4245,35.6064
4,24.7466,1026.9021,32.0453,0.1982,1.5008,88.9442
5,25.5856,1115.1917,33.3945,0.1608,1.4676,30.5779
6,24.3714,1052.2860,32.4390,0.1792,1.4470,25.9827
7,24.8285,1168.3604,34.1813,0.0582,1.4586,85.5091
8,28.4285,1317.9483,36.3036,0.0368,1.4853,49.5252


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
stacked_MLM = stack_models(tuned_MLM)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,136142679123707168524612219746873603128846603294898913280000.0000,865513352039438032143212351480734453903907374217632065873746904221891711708321032664875722866451561889745899346294996992.0000,930329700718749436679027036417707157714768430209282739273728.0000,-668640608046708053540430338904990257684060162781389941747322341525043153137556812769543260022388502440046411084988416.0000,53.9497,147082413484548844668936979214010845351987007712800920305664.0000
1,598025594838585689013578723250300509296094974962561402705805312.0000,13385615869261751682990527514816475721173473405255926932736419717461830745485561648861546534439275461663033835810708684487524352.0000,3658635793470258931211013762240770584034060520991297825128054784.0000,-10575250301748982464994722213735056016690961866714544766149471053396440269985351917098592650676024975728929189121126156992512.0000,59.8185,120199981766551330493360973066672009652392354296760316203106304.0000
2,127719483079319277771136320689637378308114354002665035268096.0000,1642602430040019593910682632403775780918030122301218322920667839732010362738747389447175924065271519302600947017361391616.0000,1281640522939259982919464984297375578497446978845086670913536.0000,-1362853339328721919276416408668072179064842495445777036345583156219466130742130505227338857852204054378589299582959616.0000,53.4234,21736021995551956407211942784738562249942215913676440862720.0000
3,360343725632571917608470263504423390483264854465591849779200.0000,6591616193646733466305664141055895015738442707651173958542811663677329530892080757344231969954191757535192410669119963136.0000,2567414301129977908017792806370738510577480114046995255001088.0000,-5021665600677064469799798182775801118420038579044826609866400663765328043747970465688411405724321660319236935565443072.0000,52.1923,1472422427991306611306938280643484127439473811363468922585088.0000
4,45022840525679486867176498667300322959680693043075127181312.0000,160830227304457504700394225274897133291416070460669715248513864734878142461034644370114085357649457943756602847780667392.0000,401036441367187358368098648858328694632084419232553441951744.0000,-125580636140985257961307292830637086278502446365000256884265411725780636886793485389910622541262919554701098611638272.0000,50.9915,6818025443158624545220191027428781247967984672337416945664.0000
5,2391063235860813619884390129247662434715330720926670868447232.0000,1111369902938626068677219763156204332731190944944470270884036025263195455805211256689572475261601797071031571691010053373952.0000,33337214984737794928392073815332916615901719550330890379853824.0000,-836276630719047052843915062366544998857635426126078602749395699736957691326895045540503057971783684696935157731006873600.0000,51.1078,931873341531988135206912871628610892401945136903596858671104.0000
6,28463236425411180446272389048927893823684063935816643117056.0000,40872575727585029722679302428703856315758959685578910225047014929163406788205836003307758367069200306299695565870465024.0000,202169670642223254269104742732554603234622978061010068307968.0000,-31879424570546352594277757207139330931566417511286998592869230711135832176263188432756831372029347399965126495830016.0000,58.3015,132187756315563997458002470122452998640225632564860157952000.0000
7,116262533577896766420102073183181333650329894303031351050240.0000,2616220415680541320732037153763918199678742164367647368100955891895485897747105983186410685307624953621716661014485794816.0000,1617473466762450905126639901564141059135565090023202908798976.0000,-2108949294709611416301140263609535254547105023350625854366625121241098327341623871843062087176169746076063317683601408.0000,44.7320,13050077236266121309890765946418246245482896991424645955584.0000
8,80263121954370708440016681980913765756928419621983450099089408.0000,210893377822836945109956913557449092137017772592353933923153915115508900244142750023538273582661321974378465827296074507747328.0000,459231290117340482196917601221768382711419436980233158388

Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
# check leaderboard
get_leaderboard()

Processing:   0%|          | 0/66 [00:00<?, ?it/s]

,Model Name,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
Index,,,,,,,,
0,Linear Regression,(TransformerWrapperWithInverse(transformer=Tar...,2.682767e+62,3.910066e+127,2.224686e+63,-3.085276e+124,54.3037,1.011223e+62
1,Lasso Regression,(TransformerWrapperWithInverse(transformer=Tar...,3.181800e+01,1.578237e+03,3.969580e+01,-2.267000e-01,1.6259,4.567410e+01
2,Ridge Regression,(TransformerWrapperWithInverse(transformer=Tar...,7.664910e+01,5.618077e+04,2.181173e+02,-4.305550e+01,2.1262,9.598210e+01
3,Elastic Net,(TransformerWrapperWithInverse(transformer=Tar...,3.054810e+01,1.476775e+03,3.839680e+01,-1.480000e-01,1.5759,4.447430e+01
4,Least Angle Regression,(TransformerWrapperWithInverse(transformer=Tar...,2.766070e+01,1.280258e+03,3.575840e+01,3.300000e-03,1.5266,4.459120e+01
...,...,...,...,...,...,...,...,...
60,Random Forest Regressor,(TransformerWrapperWithInverse(transformer=Tar...,2.575460e+01,1.189603e+03,3.445730e+01,7.350000e-02,1.4845,4.401240e+01
61,Gradient Boosting Regressor,(TransformerWrapperWithInverse(transformer=Tar...,2.649980e+01,1.207462e+03,3.472580e+01,6.010000e-02,1.4636,4.040880e+01
62,Extra Trees Regressor,(TransformerWrapperWithInverse(transformer=Tar...,2.549640e+01,1.224260e+03,3.492330e+01,4.610000e-02,1.5249,4.349750e+01


In [10]:
# check leaderboard
lb = get_leaderboard()

# select top model
best_model_MLM = lb.iloc[0]['Model']

best_model_MLM = finalize_model(best_model_MLM)

Processing:   0%|          | 0/66 [00:00<?, ?it/s]

In [11]:
# MLM 예측값을 데이터에 추가
data['MLM_predictions'] = predict_model(best_model_MLM, data=data)['prediction_label']

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,382037709376903042304186307907711674282187113786759761100800.0000,196136464469993855464347657579094810329407513634836458452131337587103340476887293973917963488871632911104483662013667475456.0000,14004872883035885645181949994346068447248917046695646745067520.0000,-153972647686637620320465337332603171994046750790920840611005209904186676674439171454637163687869985665603171178522869760.0000,12.6282,17990402480698840060942508984504403073225270415617159069696.0000


In [ ]:
# HLM을 위한 setup (MLM 예측값 포함)
setup_data_HLM = setup(data=data,
                       target='HLM',
                       ignore_features=['id', 'SMILES'],
                       remove_outliers=True,
                       normalize=True,
                       feature_selection=True,
                       use_gpu=True,
                       session_id=123,
                       verbose=True
                       )

In [ ]:
# HLM 모델 학습, 튜닝, 블렌딩, 스택킹
top_5_HLM = compare_models(n_select=5)

In [ ]:
tuned_HLM = [tune_model(model, optimize='RMSE', n_iter=10) for model in top_5_HLM]

In [ ]:
blended_HLM = blend_models(tuned_HLM, optimize='RMSE')

In [ ]:
stacked_HLM = stack_models(tuned_HLM)

In [ ]:
# check leaderboard
get_leaderboard()

In [ ]:
# check leaderboard
lb = get_leaderboard()

# select top model
best_model_HLM = lb.iloc[0]['Model']

best_model_HLM = finalize_model(best_model_HLM)

## submit csv 생성

In [ ]:
test_data = pd.read_csv('test_pycaret.csv')

# MLM 예측하기 위한 특성 데이터 준비
features_for_MLM_prediction = test_data.drop(columns=['id', 'SMILES', 'MLM', 'HLM'])

# MLM 예측
predicted_MLM = predict_model(best_model_MLM, data=features_for_MLM_prediction)['prediction_label'].values
test_data['MLM'] = predicted_MLM

# HLM 예측하기 위한 특성 데이터 준비 (MLM이 추가된 데이터를 사용)
features_for_HLM_prediction = test_data.drop(columns=['id', 'SMILES', 'HLM'])

# HLM 예측
predicted_HLM = predict_model(best_model_HLM, data=features_for_HLM_prediction)['prediction_label'].values
test_data['HLM'] = predicted_HLM

# submission.csv 파일 생성
submission = test_data[['id', 'MLM', 'HLM']]
submission.set_index('id', inplace=True)
submission.to_csv('submission_pycaret_tuned.csv')